In [7]:
import pandas as pd
import cudf
%cd /data/dhk/physionet.org/files/mimiciv/2.2/hosp

/data/dhk/physionet.org/files/mimiciv/2.2/hosp


In [8]:
df_dh = cudf.read_csv("process_dohun.csv")
df_dh = df_dh.drop('subject_id', axis=1)
df_dh = df_dh.drop('hadm_id', axis=1)
df_dh


y = df_dh['hospital_expire_flag'] 
X = df_dh.drop('hospital_expire_flag', axis=1) # 'label' 열을 제외한 나머지 열을 X로 사용


In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from cuml.model_selection import train_test_split

In [10]:
model = XGBClassifier(tree_method='gpu_hist')

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=42)

X_train = X_train.to_cupy()
X_test = X_test.to_cupy()
y_train = y_train.to_cupy()
y_test = y_test.to_cupy()

# 매개변수 그리드 정의
param = {
  'max_depth': [3, 4, 5],
  'random_state': [0, 42],
  'learning_rate': [0.1, 0.01, 0.001],
  'n_estimators': [100, 500, 1000],
  'subsample': [0.8, 0.9, 1.0],
  'colsample_bytree': [0.8, 0.9, 1.0]
}
# GridSearchCV 객체 생성
grid_search = GridSearchCV(model, param_grid=param, cv=5, refit=True, n_jobs=-1, return_train_score=True, verbose=2, error_score='raise')
# 훈련 데이터로 모델 학습 및 최적 매개변수 탐색
best_model = grid_search.fit(X_train.get(), y_train.get())
# 최적 매개변수 출력
print("최적 매개변수:", grid_search.best_params_)

y_pred = best_model.predict(X_test.get())

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [01:34:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [01:34:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [01:34:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_m

[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=100, random_state=42, subsample=0.8; total time=  31.5s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=500, random_state=42, subsample=1.0; total time= 3.9min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=1000, random_state=42, subsample=0.8; total time= 6.9min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=4, n_estimators=100, random_state=42, subsample=0.9; total time= 2.2min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=4, n_estimators=500, random_state=42, subsample=1.0; total time= 4.3min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=100, random_state=0, subsample=0.8; total time= 2.2min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=100, random_state=0, subsample=0.8; total time= 2.2min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=1

/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [06:21:57] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [06:21:59] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [06:22:03] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_m

[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=100, random_state=0, subsample=0.8; total time=  30.8s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=500, random_state=42, subsample=1.0; total time= 3.8min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=1000, random_state=0, subsample=1.0; total time= 6.8min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=4, n_estimators=100, random_state=42, subsample=0.9; total time= 2.1min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=4, n_estimators=500, random_state=42, subsample=0.9; total time= 4.5min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=4, n_estimators=1000, random_state=42, subsample=0.9; total time= 7.8min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=500, random_state=0, subsample=0.9; total time= 4.4min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=1

...

/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:04:28] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:04:31] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:04:37] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_m

[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=100, random_state=0, subsample=1.0; total time= 2.2min
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=500, random_state=0, subsample=1.0; total time= 4.9min
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=1000, random_state=42, subsample=0.8; total time= 7.6min
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=3, n_estimators=100, random_state=42, subsample=0.8; total time= 2.0min
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=3, n_estimators=500, random_state=42, subsample=0.8; total time= 4.2min
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=3, n_estimators=1000, random_state=42, subsample=0.9; total time= 6.9min
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=4, n_estimators=100, random_state=42, subsample=0.9; total time= 2.1min
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=4, 

/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:04:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:05:32] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=4, n_estimators=1000, random_state=0, subsample=1.0; total time= 7.3min
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=100, random_state=42, subsample=0.8; total time= 2.1min
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=500, random_state=42, subsample=0.8; total time= 4.7min
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=1000, random_state=42, subsample=0.8; total time= 7.9min
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=3, n_estimators=100, random_state=42, subsample=0.9; total time= 2.0min
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=3, n_estimators=500, random_state=42, subsample=0.9; total time= 4.2min
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=3, n_estimators=1000, random_state=42, subsample=0.9; total time= 6.8min
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=4,

/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:11:24] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [15]:
import cuml
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

cu_score = cuml.metrics.accuracy_score( y_test, y_pred )
print(cu_score)

# 정확도 계산
accuracy = accuracy_score(y_test.get(), y_pred)
print("Accuracy:", accuracy)

# F1 스코어 계산
f1 = f1_score(y_test.get(), y_pred)
print("F1 Score:", f1)

0.9360584020614624
Accuracy: 0.9360583751195618
F1 Score: 0.9375277581251956


In [16]:
import pandas as pd

# randomForest_y_pred를 Pandas DataFrame으로 변환
y_pred_df = pd.DataFrame(y_pred)

# CSV 파일로 저장
y_pred_df.to_csv('xgboost_y_pred.csv', index=False, header=False)